#Data Cleaning
##What is data cleaning?
Data cleaning is the process of identifying and correcting errors or inconsistencies in a dataset to ensure its accuracy, completeness, and reliability for analysis. This involves removing or fixing missing, incorrect, duplicated, or irrelevant data.

##Why do we need data cleaning?
Data cleaning is essential to ensure that analysis and models are based on accurate, consistent, and reliable data. Without it, results can be misleading or incorrect.

##What does data cleaning for data science include?
Data cleaning for data science includes-

1.Handling missing values and duplicates
2.Standardizing data



#1. Handling Missing values
Handling missing values is crucial in data science as they can distort dataset integrity, leading to biased analyses and unreliable predictions. Missing data affects statistical calculations and can cause machine learning algorithms to fail. Addressing them through strategies like imputation or deletion ensures accurate analysis, improves model performance, and enhances decision-making.









In [2]:
!pip install kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"riyabangera","key":"d5a92cf8021850a75165fe9784b5720b"}'}

In [4]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [5]:
!kaggle datasets download -d maxhorowitz/nflplaybyplay2009to2016

Dataset URL: https://www.kaggle.com/datasets/maxhorowitz/nflplaybyplay2009to2016
License(s): unknown
 97% 265M/274M [00:03<00:00, 66.1MB/s]
100% 274M/274M [00:03<00:00, 75.1MB/s]


In [6]:
!ls


nflplaybyplay2009to2016.zip  sample_data


In [7]:
!unzip nflplaybyplay2009to2016.zip

Archive:  nflplaybyplay2009to2016.zip
  inflating: NFL Play by Play 2009-2016 (v3).csv  
  inflating: NFL Play by Play 2009-2017 (v4).csv  
  inflating: NFL Play by Play 2009-2018 (v5).csv  


In [8]:
!ls

'NFL Play by Play 2009-2016 (v3).csv'  'NFL Play by Play 2009-2018 (v5).csv'   sample_data
'NFL Play by Play 2009-2017 (v4).csv'   nflplaybyplay2009to2016.zip


In [9]:
import pandas as pd
import numpy as np

# read in all our data
nfl_data = pd.read_csv("NFL Play by Play 2009-2017 (v4).csv")

# set seed for reproducibility
np.random.seed(0)

<ipython-input-9-87e9b5524a54>:5: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  nfl_data = pd.read_csv("NFL Play by Play 2009-2017 (v4).csv")


Here we can observe the missing values indicated as NaN

In [10]:
nfl_data.head()

Date      GameID  Drive  qtr  down   time  TimeUnder  TimeSecs  \
0  2009-09-10  2009091000      1    1   NaN  15:00         15    3600.0   
1  2009-09-10  2009091000      1    1   1.0  14:53         15    3593.0   
2  2009-09-10  2009091000      1    1   2.0  14:16         15    3556.0   
3  2009-09-10  2009091000      1    1   3.0  13:35         14    3515.0   
4  2009-09-10  2009091000      1    1   4.0  13:27         14    3507.0   

   PlayTimeDiff SideofField  ...    yacEPA  Home_WP_pre  Away_WP_pre  \
0           0.0         TEN  ...       NaN     0.485675     0.514325   
1           7.0         PIT  ...  1.146076     0.546433     0.453567   
2          37.0         PIT  ...       NaN     0.551088     0.448912   
3          41.0         PIT  ... -5.031425     0.510793     0.489207   
4           8.0         PIT  ...       NaN     0.461217     0.538783   

   Home_WP_post  Away_WP_post  Win_Prob       WPA    airWPA    yacWPA  Season  
0      0.546433      0.453567  0.485675  0.060758       NaN       NaN    2009  
1      0.551088      0.448912  0.546433  0.004655 -0.032244  0.036899    2009  
2      0.510793      0.489207  0.551088 -0.040295       NaN       NaN    2009  
3      0.461217      0.538783  0.510793 -0.049576  0.106663 -0.156239    2009  
4      0.558929      0.441071  0.461217  0.097712       NaN       NaN    2009  

[5 rows x 102 columns]

In [11]:
# get the number of missing data points per column
missing_values_count = nfl_data.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count[0:10]

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

It might be helpful to see what percentage of the values in our dataset were missing to give us a better sense of the scale of this problem

In [12]:
# how many total missing values do we have?
total_cells = np.product(nfl_data.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
(total_missing/total_cells) * 100

27.66722370547874

##How to deal with missing values?
For dealing with missing values, you'll need to use your intution to figure out why the value is missing. One of the most important question you can ask yourself to help figure this out is this:

Is this value missing becuase it wasn't recorded or becuase it dosen't exist?

If a value is missing becuase it doens't exist (like the height of the oldest child of someone who doesn't have any children) then it doesn't make sense to try and guess what it might be. These values you probalby do want to keep as NaN. On the other hand, if a value is missing becuase it wasn't recorded, then you can try to guess what it might have been based on the other values in that column and row.

In [13]:
# look at the # of missing points in the first ten columns
missing_values_count[0:10]

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

I can see that this column has information on the number of seconds left in the game when the play was made. This means that these values are probably missing because they were not recorded, rather than because they don't exist. So, it would make sense for us to try and guess what they should be rather than just leaving them as NA's.

On the other hand, there are other fields, like PenalizedTeam that also have lot of missing fields. In this case, though, the field is missing because if there was no penalty then it doesn't make sense to say which team was penalized. For this column, it would make more sense to either leave it empty or to add a third value like "neither" and use that to replace the NA's.

If you're doing very careful data analysis, this is the point at which you'd look at each column individually to figure out the best strategy for filling those missing values.

If you're in a hurry or don't have a reason to figure out why your values are missing, one option you have is to just remove any rows or columns that contain missing values. (Note: I don't generally recommend this approch for important projects! It's usually worth it to take the time to go through your data and really look at all the columns with missing values one-by-one to really get to know your dataset.)

If you're sure you want to drop rows with missing values, pandas does have a handy function, dropna() to help you do this.

In [14]:
nfl_data.dropna()

Empty DataFrame
Columns: [Date, GameID, Drive, qtr, down, time, TimeUnder, TimeSecs, PlayTimeDiff, SideofField, yrdln, yrdline100, ydstogo, ydsnet, GoalToGo, FirstDown, posteam, DefensiveTeam, desc, PlayAttempted, Yards.Gained, sp, Touchdown, ExPointResult, TwoPointConv, DefTwoPoint, Safety, Onsidekick, PuntResult, PlayType, Passer, Passer_ID, PassAttempt, PassOutcome, PassLength, AirYards, YardsAfterCatch, QBHit, PassLocation, InterceptionThrown, Interceptor, Rusher, Rusher_ID, RushAttempt, RunLocation, RunGap, Receiver, Receiver_ID, Reception, ReturnResult, Returner, BlockingPlayer, Tackler1, Tackler2, FieldGoalResult, FieldGoalDistance, Fumble, RecFumbTeam, RecFumbPlayer, Sack, Challenge.Replay, ChalReplayResult, Accepted.Penalty, PenalizedTeam, PenaltyType, PenalizedPlayer, Penalty.Yards, PosTeamScore, DefTeamScore, ScoreDiff, AbsScoreDiff, HomeTeam, AwayTeam, Timeout_Indicator, Timeout_Team, posteam_timeouts_pre, HomeTimeouts_Remaining_Pre, AwayTimeouts_Remaining_Pre, HomeTimeouts_Remaining_Post, AwayTimeouts_Remaining_Post, No_Score_Prob, Opp_Field_Goal_Prob, Opp_Safety_Prob, Opp_Touchdown_Prob, Field_Goal_Prob, Safety_Prob, Touchdown_Prob, ExPoint_Prob, TwoPoint_Prob, ExpPts, EPA, airEPA, yacEPA, Home_WP_pre, Away_WP_pre, Home_WP_post, Away_WP_post, Win_Prob, WPA, airWPA, ...]
Index: []

[0 rows x 102 columns]

it looks like that's removed all our data! This is because every row in our dataset had at least one missing value. We might have better luck removing all the columns that have at least one missing value instead.



In [15]:
# remove all columns with at least one missing value
columns_with_na_dropped = nfl_data.dropna(axis=1)
columns_with_na_dropped.head()

Date      GameID  Drive  qtr  TimeUnder  ydstogo  ydsnet  \
0  2009-09-10  2009091000      1    1         15        0       0   
1  2009-09-10  2009091000      1    1         15       10       5   
2  2009-09-10  2009091000      1    1         15        5       2   
3  2009-09-10  2009091000      1    1         14        8       2   
4  2009-09-10  2009091000      1    1         14        8       2   

   PlayAttempted  Yards.Gained  sp  ...  AwayTeam  Timeout_Indicator  \
0              1            39   0  ...       TEN                  0   
1              1             5   0  ...       TEN                  0   
2              1            -3   0  ...       TEN                  0   
3              1             0   0  ...       TEN                  0   
4              1             0   0  ...       TEN                  0   

   posteam_timeouts_pre HomeTimeouts_Remaining_Pre  \
0                     3                          3   
1                     3                          3   
2                     3                          3   
3                     3                          3   
4                     3                          3   

   AwayTimeouts_Remaining_Pre  HomeTimeouts_Remaining_Post  \
0                           3                            3   
1                           3                            3   
2                           3                            3   
3                           3                            3   
4                           3                            3   

   AwayTimeouts_Remaining_Post  ExPoint_Prob  TwoPoint_Prob  Season  
0                            3           0.0            0.0    2009  
1                            3           0.0            0.0    2009  
2                            3           0.0            0.0    2009  
3                            3           0.0            0.0    2009  
4                            3           0.0            0.0    2009  

[5 rows x 37 columns]

In [16]:
# just how much data did we lose?
print("Columns in original dataset: %d \n" % nfl_data.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

Columns in original dataset: 102 

Columns with na's dropped: 37


In [19]:
nfl_data.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
407683    False
407684    False
407685    False
407686    False
407687    False
Length: 407688, dtype: bool